# Chapter 7 - Training Flair Embeddings

This Jupyter notebook provides a resource to help you follow the code examples from the book more easily. The notebook covers all practical code snippets and exercises found in: Chapter 7 - Training Flair Embeddings.

## Training Flair embeddings on the world’s smallest language

### Preparing the dictionary

In [18]:
from flair.data import Dictionary

dictionary = Dictionary.load('chars')

In [19]:
from flair.data import Dictionary

dictionary = Dictionary()
toki_pona_symbols = 'ptksmnljwaeiou'
toki_pona_symbols += toki_pona_symbols.upper()

for c in toki_pona_symbols + '?. ':
    dictionary.add_item(c)

### Preparing the corpus

In [20]:
import requests

response = requests.get("https://git.io/J1dgd")

sentences = response.text.splitlines()
one_tenth_corp_len = int(len(sentences)/10)

test, valid, train = (
    sentences[:one_tenth_corp_len],
    sentences[one_tenth_corp_len:one_tenth_corp_len*2],
    sentences[one_tenth_corp_len*2:])

In [21]:
from tempfile import TemporaryDirectory
from os.path import join
from os import mkdir

dataset_dir_obj = TemporaryDirectory()
dataset_dir = dataset_dir_obj.name
train_dir = join(dataset_dir, 'train')
mkdir(train_dir)

with open(join(dataset_dir, "test.txt"), "w") as file:
    file.write(' '.join(test))

with open(join(dataset_dir, "valid.txt"), "w") as file:
    file.write(' '.join(valid))

with open(join(train_dir, "train_split_1"), "w") as file:
    file.write(' '.join(train))

In [22]:
from flair.trainers.language_model_trainer import TextCorpus

corpus = TextCorpus(dataset_dir,
                    dictionary,
                    forward=True,
                    character_level=True)

2021-12-01 04:48:34,283 read text file with 1 lines
2021-12-01 04:48:34,291 read text file with 1 lines


### Training the language model

In [23]:
from flair.models import LanguageModel
from flair.trainers.language_model_trainer import (
    LanguageModelTrainer)

language_model = LanguageModel(dictionary,
                               is_forward_lm=True,
                               hidden_size=64,
                               nlayers=1)

trainer = LanguageModelTrainer(language_model, corpus)
trainer.train('forward_model_directory',
              sequence_length=25,
              mini_batch_size=10,
              max_epochs=100)

2021-12-01 04:48:34,343 read text file with 1 lines
2021-12-01 04:48:34,349 shuffled
2021-12-01 04:48:34,372 Sequence length is 25
2021-12-01 04:48:34,375 Split 1	 - (04:48:34)
2021-12-01 04:48:35,168 | split   1 /  1 |   100/  124 batches | ms/batch  7.92 | loss  2.34 | ppl    10.33
2021-12-01 04:48:35,358 0 seconds for train split 1
2021-12-01 04:48:35,391 best loss so far 10000.00
2021-12-01 04:48:35,666 ('\nluko kakama. la? li. kapani kmakan iloko seKen okawakama. koli ulasoka keli kakansa. mama. kalasa. tanka li nini mi poma lon pinama. mi. pasannpa. pikenpa? nakalasokajan wanni kasama li. anawalala kikanpa jann epin sona nina. pani. kalal? ni.pa awanapo. seme? li? li toka. ekankawae kala okin lela. ninu lon kali ninasa. eni lini. kalaasa kapaka kanomepa. pikonpa silo okinsi pikelime? ni<unk> keli ni oni. jomen sinaawa. ena. kaekopo. kinaja li. nijan<unk> Nukakemanpa. kali sike. toki kawalalaa. kae mi panapo poki. kala esana. lipana naseka. wanasa jan. nijan anala laPa. kome. kala

2021-12-01 04:48:41,135 -----------------------------------------------------------------------------------------
2021-12-01 04:48:41,137 | end of split   1 /  1 | epoch   5 | time:  1.25s | valid loss  1.36 | valid ppl     3.89 | learning rate 20.0000
2021-12-01 04:48:41,138 -----------------------------------------------------------------------------------------
2021-12-01 04:48:41,146 Epoch time: 1.31
2021-12-01 04:48:41,168 read text file with 1 lines
2021-12-01 04:48:41,173 shuffled
2021-12-01 04:48:41,194 Sequence length is 25
2021-12-01 04:48:41,197 Split 1	 - (04:48:41)
2021-12-01 04:48:41,938 | split   1 /  1 |   100/  124 batches | ms/batch  7.39 | loss  1.15 | ppl     3.16
2021-12-01 04:48:42,113 0 seconds for train split 1
2021-12-01 04:48:42,161 best loss so far  1.36
2021-12-01 04:48:42,529 ('\n ni li kama. sina pini? ni li pimeja ni<unk> waso muli ni li kiwen ni li pini. kule li ni li ike sina. sina tawa sina. koki li pini ke pi. ni<unk> ni<unk> soweni. ni. sinsini. ni l

2021-12-01 04:48:48,147 best loss so far  1.23
2021-12-01 04:48:48,482 ('\n ni<unk> kalama suli ni li pini. tenpo insa mi pimi ni li pini. ni. sina kai li ken pi poki. ni. ko pimi. tenpo li kipisi. kule mi. jan ike ma sina. jan ma tomo ken pona. jan ike insa tomo mi. o ken tan tomo poki li pona. pipi pi moki e moku e ni<unk> lipu ni li pini. sina jo e kon. mani ni li ken pi ni li uta sina. jan pi kalama tan kasi li wile telo ni li li pini en  kon e ikun lon insa insa. sina sona. mi. tomo ni li pakili. meli mi. sina pini. ni li taki pi sowi anu li sewi kepeken tomo ko. uta sina jaki sina. nimi ni li pona. kipisi e lon sina. kin tawa sina li pini. sina ken pi toki e sona. kipisi li ike anu li jan li toki ni li sina. ko ali ni li jaki mute tan insi ni li sinso. nimi li wile e pipi ali pimeja Osuma tomo li tan poki ni li pini li wile pi kiwen Anki. ona li wile e kiwen li pini. pipini jaki. ni. nena. tenpo. pini. sina kiken ken li awen pini. kiwen mute ni ni li pini. ni li pipi tawa wile lo

2021-12-01 04:48:55,857 -----------------------------------------------------------------------------------------
2021-12-01 04:48:55,859 | end of split   1 /  1 | epoch  14 | time:  1.60s | valid loss  1.27 | valid ppl     3.56 | learning rate 20.0000
2021-12-01 04:48:55,860 -----------------------------------------------------------------------------------------
2021-12-01 04:48:55,867 Epoch time: 1.69
2021-12-01 04:48:55,892 read text file with 1 lines
2021-12-01 04:48:55,898 shuffled
2021-12-01 04:48:55,926 Sequence length is 25
2021-12-01 04:48:55,931 Split 1	 - (04:48:55)
2021-12-01 04:48:56,886 | split   1 /  1 |   100/  124 batches | ms/batch  9.50 | loss  1.04 | ppl     2.84
2021-12-01 04:48:57,085 1 seconds for train split 1
2021-12-01 04:48:57,119 best loss so far  1.21
2021-12-01 04:48:57,419 ('\n telo ni li pini li. ijo pi insa pini. kalama kalama pi. pimeja kipisi ni. kipisi. tomo e lipu muli tan ma. tenpo ike insa tomo kiwilo li jelo. nimi ni li ipini. ni. nimi uta pi ni

2021-12-01 04:49:03,040 ('\n ni li pakala e ni<unk> tenpo. tenpo pini li pana e ona. kan pona tan ni<unk> toki  ni li pona tawa pona. kili insa pini li pona. kulupu sona kamo kepeken poki ni li pana e nimi ni li pana anu ni lon poki. pi ni li pana e ken pona tawa tomo ona. mi. jakin telo kinsa toki e kiweki e kiwen li pana e ma. ko ni li ken kepeken en ni li pona ana Menta? toki li kama pini. nimi wile e ona. tenpo piki li ike li ike toki. kun taso li jo e poki. kon pimeja jan pona. kinsa monsuta mi. tenpo kini li alasa e kini. kinse ni li toki. pini pi poki pi wawa lawa sina. poki. kinsa. jelo. ni. ni li kama sona e kijetesantakanu ni li pona tawa tan ni. ni li kepeken. ona li pini. toki kon li ken pana. mi kepeken. e ni<unk> toki Leme? nimi nasa. soweli mi li pona tawa pi la ona. ona li ike li toki li pona. jan pona tawa jan Kama kulupu ni li pona anu kasi. jan Jkantake li ike ni li pini li pakala e kule pini. ni li jo e mani. ni li ken pona. ni<unk> kiwen pi sike jaki jelo. kiwen ke

...

2021-12-01 04:51:17,962 -----------------------------------------------------------------------------------------
2021-12-01 04:51:17,965 | end of split   1 /  1 | epoch  79 | time:  1.21s | valid loss  1.00 | valid ppl     2.71 | learning rate 0.0781
2021-12-01 04:51:17,966 -----------------------------------------------------------------------------------------
2021-12-01 04:51:17,973 Epoch time: 1.26
2021-12-01 04:51:17,993 read text file with 1 lines
2021-12-01 04:51:17,997 shuffled
2021-12-01 04:51:18,018 Sequence length is 25
2021-12-01 04:51:18,020 Split 1	 - (04:51:18)
2021-12-01 04:51:18,780 | split   1 /  1 |   100/  124 batches | ms/batch  7.58 | loss  0.76 | ppl     2.14
2021-12-01 04:51:18,952 0 seconds for train split 1
2021-12-01 04:51:18,982 best loss so far  1.00
2021-12-01 04:51:19,242 ('\n moli tawa mi li pona lukin. oko mi li pona tawa sina. soweli li sin e sitelen toki li kama jan loje anu lili li pakala kepeken e sike. mi wile telo li sona e sijelo o kute e mun so

2021-12-01 05:31:12,469 ('\n sijelo sina li jo e sijelo pi jan pona e jan Sakulasa kiwen sike ala. nena sewi en ni<unk> ona la kama tan ni<unk> sina ken toki e ilo seli ala. lupa sina li laso jelo. tomo li kama ni. ma ni li sona ala la mi wile sona ala e kalama sewi. o pakala e ona ni. mi pona tawa? ma Ken li ike alama pi toki pona. sina ken ala toki mute. poki li sona lon nasin sama seli e pali mi la mi moku e kon nanpa li kama wawa. sina moku pona lukin li sona tawa mi e litejaki tawa mi. uta sina li kama pini Lawa seme? seli li jo e ilo kepeken ala lon tomo lipu meli. monsi Piseme lete li open. waso li pona tawa sijelo. o walo. jan pali sima mute li Sasun li moli ala. kulupu ali li pakala e toki lukin? nena li sona e ni<unk> sina sijelo sina li ken ala sona ponako la o pona e ona. pipi pi jan utala tawa MAkama tan ni<unk> mi ken ala seme? seli ni li pona tawa kepeken kasi tan lupa. waso mute li sitelen e alasilo musi e poki la sina jaki tan ni<unk> kulupu ala. jan JukuAjetta loje mu

2021-12-01 06:25:14,243 -----------------------------------------------------------------------------------------
2021-12-01 06:25:14,246 | end of split   1 /  1 | epoch  88 | time:  1.56s | valid loss  1.00 | valid ppl     2.71 | learning rate 0.0781
2021-12-01 06:25:14,247 -----------------------------------------------------------------------------------------
2021-12-01 06:25:14,253 Epoch time: 1.63
2021-12-01 06:25:14,274 read text file with 1 lines
2021-12-01 06:25:14,279 shuffled
2021-12-01 06:25:14,300 Sequence length is 25
2021-12-01 06:25:14,304 Split 1	 - (06:25:14)
2021-12-01 06:25:15,196 | split   1 /  1 |   100/  124 batches | ms/batch  8.91 | loss  0.76 | ppl     2.14
2021-12-01 06:25:15,394 1 seconds for train split 1
2021-12-01 06:25:15,429 best loss so far  1.00
2021-12-01 06:25:15,720 ('\n tenpo suli tawa Ipalaka li loje en sama jan mute e ijo ala. jan sewi Wute li moku e ko mute la mi ken ala toki tawa mu. ni li alasa e mani. kala li mama mi. nasin Onsa li jo e moku

2021-12-01 06:25:22,933 best loss so far  1.00
2021-12-01 06:25:24,674 ('\n mama kute li utala e mama. jan. soweli ni li jo e sike musi la kalama sona sona e poki jaki e ni<unk> open li pona tawa jan sewi. telo pimeja li kama ala. o lete lon insa anu ni la kala Seteso la jan seme mute li lon insa luka. supa seli mute lon mi li pona e kili sina? mi wile pilin e ni<unk> mi pona ala lon tenpo mute. jan ni li jo e len sona kepeken kasi. linja li pona tawa Jusepeken pi kala ni. uta ona li pini e ni<unk> mi pona lukin. o lon insa mi la mi mute pi jan li olin e ni<unk> mi lukin. mi pali e toki mun esun. tenpo pimeja en supa sina li musi la jan Kan tawa ni li pakala e ona. mi mute li sona e sitelen toki mama sina? mi sona ala esun e tomo seli lon ma kepeken e oko oko pini. toki MEja li lon insa mama Sonku. kulupu sina li pona sinpin tan ni<unk> sina pini e ko sala loje nasa sike. moku Lopi li pakala e ma. ilo jaki li pona lukin. sina Manja. tenpo ni lukin anu kon. sina lape li nasa mute ala. w

2021-12-01 08:13:16,324 -----------------------------------------------------------------------------------------
2021-12-01 08:13:16,327 | end of split   1 /  1 | epoch  97 | time: 3236.27s | valid loss  1.00 | valid ppl     2.71 | learning rate 0.0195
2021-12-01 08:13:16,329 -----------------------------------------------------------------------------------------
2021-12-01 08:13:16,824 Epoch time: 3236.82
2021-12-01 08:13:16,874 read text file with 1 lines
2021-12-01 08:13:16,881 shuffled
2021-12-01 08:13:16,919 Sequence length is 25
2021-12-01 08:13:16,925 Split 1	 - (08:13:16)
2021-12-01 08:13:18,140 | split   1 /  1 |   100/  124 batches | ms/batch 12.12 | loss  0.76 | ppl     2.13
2021-12-01 08:13:18,453 1 seconds for train split 1
2021-12-01 08:13:18,515 best loss so far  1.00
2021-12-01 08:13:19,058 ('\n selo ona li lon seme? tenpo ansa lukin li lon supa. mi pona ala kon ni la ona li pana e telo ala sama ante. meli li toki. tenpo pimeja ni la e mije ala pimeja pi nimi pona. se

In [24]:
t = language_model.generate_text(number_of_characters=40)[0]
print(t)


 esun li kalama musi utala kepeken telo 


### Using custom embeddings on downstream tasks

In [25]:
from flair.embeddings import FlairEmbeddings

fw = FlairEmbeddings('forward_model_directory/best-lm.pt')

### Performing intrinsic evaluation on custom Flair embeddings

In [26]:
from flair.embeddings import FlairEmbeddings
from flair.data import Sentence

synonym_1 = Sentence('lukin')
synonym_2 = Sentence('oko')
rand_word = Sentence('jan')

fw = FlairEmbeddings('forward_model_directory/best-lm.pt')
fw.embed(synonym_1)
fw.embed(synonym_2)
fw.embed(rand_word)

embedding_syn_1 = synonym_1[0].embedding.tolist()
embedding_syn_2 = synonym_2[0].embedding.tolist()
embedding_rnd_wrd = rand_word[0].embedding.tolist()

In [27]:
from sklearn.metrics.pairwise import cosine_similarity as sim

s_synonym = sim([embedding_syn_1], [embedding_syn_2])[0][0] 
s_rand_1 = sim([embedding_syn_1], [embedding_rnd_wrd])[0][0]
s_rand_2 = sim([embedding_syn_2], [embedding_rnd_wrd])[0][0]

In [28]:
print(s_synonym , s_rand_1, s_rand_2)

0.5559661163545663 0.41215033631546194 0.5142904744116392
